# The packaages we need are really simple. For the scrapping we only need python requests and json. For the data "storing" we will use pandas dataframes and csv

In [0]:
import requests
import json
import time
import pandas as pd
import time
import datetime
import csv


# Here is the first function we create. We collect the html code of a single post and collect every data we want about it :

In [0]:
def Data_From_Posts(number_of_posts,outfile_name):
    
    with open(outfile_name+'_posts.json', 'r') as f:
        arr = json.loads(f.read()) # load json data from previous step
    
    print("Opening of "+outfile_name+"_posts.json successful")  
    posts_data = []
    coms= []
    url_imgs= []
    hashtags= []
    number_of_posts_limit=number_of_posts
    number_of_posts=0
    successful_requests=0
    failed_requests=0
    kicked_requests=0
    print("Start retrieving posts data")
    for item in arr:
        if(number_of_posts<number_of_posts_limit):
            shortcode = item['shortcode']
            url = "https://www.instagram.com/p/{0}/?__a=1".format(shortcode)
            while True:
                try:
                    r = requests.get(url,timeout = 5)
                    time.sleep(500)
                except Exception as e:
                    continue
                break
            if ( r.status_code==200):
                number_of_posts+=1
                successful_requests+=1
                data = json.loads(r.text)
                #print(data)
                try:
                    location = data['graphql']['shortcode_media']['location']['name'] # get location for a post

                except:
                     location = '' # if location is NULL


                likes=data['graphql']['shortcode_media']['edge_media_preview_like']['count'] #ok

                try :
                    number_coms=data['graphql']['shortcode_media']['edge_media_to_parent_comment']['count'] #ok
                except :
                    number_coms=data['graphql']['shortcode_media']['edge_media_to_comment']['count']

                coms=[]

                try :
                    for nodes in data['graphql']['shortcode_media']['edge_media_to_parent_comment']['edges']:
                        coms.append(nodes['node']['text'])
                except:
                    try : 
                        for nodes in data['graphql']['shortcode_media']['edge_media_to_comment']['edges']:
                            coms.append(nodes['node']['text'])

                    except : 
                        coms.append("")

                url_img=data['graphql']['shortcode_media']['display_url'] # ok 
                if (len(data['graphql']['shortcode_media']['edge_media_to_caption']['edges'])>0):
                    text=data['graphql']['shortcode_media']['edge_media_to_caption']['edges'][0]['node']['text'] #ok
                else :
                    text=""
                hashtags=Hashtags_From_Text(text)
                posts_data.append({'Shortcode': shortcode,'Likes': likes,'Hashtags': hashtags,'Number of com\'s' : number_coms ,'Com\'s': coms ,'Text': text,'Url_img' : url_img,'Location': location})
                r.status_code=404
            else :
                failed_requests+=1
        time.sleep(10)
                
    #print("Requests successful : ",successful_requests)
    #print("Requests failed : ",failed_requests)
    print("Kicked requests : ",kicked_requests)
    with open(outfile_name+'_posts_data.json', 'w') as outfile:
        json.dump(posts_data, outfile) # save to json
    print(outfile_name+'_posts_data.json has been created')
    df=pd.read_json(outfile_name+"_posts_data.json")
    df.to_csv(outfile_name+'_data_results.csv')
    print(outfile_name+'_data_results.csv has been created')


# Here is the function that collect datas for posts containing a certain hashtag (see report for more information)

In [0]:
def Posts_Data_From_Tag(tag,number_of_posts,outfile_name,minimum_likes,request_time_limit):
    end_cursor = '' # empty for the 1st page
    print("Start retrieving posts data from hashtag: ",tag,)
    print("Number of posts requested: ",number_of_posts)
    successful_requests=0
    failed_requests=0
    kicked_requests=0
    kicked_limit=10
    posts_data=[]
    currentDT = datetime.datetime.now()
    currentDT_month=str(currentDT.month)
    currentDT_day=str(currentDT.day)
    pages_retrieved = 0
    posts_retrieved=0
    if (len(currentDT_month)==1):
        currentDT_month="0"+currentDT_month
    if (len(currentDT_day)==1):
        currentDT_day="0"+currentDT_day
    
    currentDT=currentDT_day+currentDT_month+str(currentDT.year)
    request_time_limit+=2
    time_limit=time.time()
    while(posts_retrieved<number_of_posts):
        if ( time.time() - time_limit > request_time_limit) : 
            print("Exceeded time request to retrieve more posts")
            break
        url = "https://www.instagram.com/explore/tags/{0}/?__a=1&max_id={1}".format(tag, end_cursor)
        kicked_times=0
        while True and kicked_times<kicked_limit:
            try:
                r = requests.get(url,timeout = 2)
            except Exception as e:
                time.sleep(1)
                kicked_requests+=1
                kicked_times+=1
                continue
            break
        if ( r.status_code == 200):
            raw_data = r.text
            data = json.loads(raw_data.strip(),strict = False)
            #print(data)
            
            edge_hashtag= data['graphql']['hashtag']
            edge_hashtag_to_media=edge_hashtag['edge_hashtag_to_media']
            
            end_cursor = edge_hashtag_to_media ['page_info']['end_cursor'] # value for the next page
            
            edges = edge_hashtag_to_media ['edges'] # list with posts
            if(posts_retrieved==0):
                tag_id = edge_hashtag['id']
                tag_name = tag
                number_of_post_with_hashtag = edge_hashtag_to_media['count']
            page_time_start=time.time()
            for node in edges:
                if(posts_retrieved<number_of_posts):
                    post=node['node']
                    likes=post['edge_liked_by']['count']
                    if(likes>=minimum_likes):
                        #print("Retrieving post number",retrieved_post_number, "from page ",i)
                        owner_id = post['owner']['id']
                        timestamp = post['taken_at_timestamp']
                        shortcode=post['shortcode']
                        try :
                            number_coms=post['edge_media_to_comment']['count'] 
                        except :
                            number_coms=post['edge_media_parent_to_comment']['count']

                        coms=[]

                        url_img=post['display_url'] 
                        edges_media=post['edge_media_to_caption']['edges']
                        if (len(edges_media)>0):
                            text=edges_media[0]['node']['text'] 
                        else :
                            text=""

                        hashtags=Hashtags_From_Text(text)
                        mentions=Mentions_From_Text(text)
                        if( post["is_video"]==False):
                            img_description=post["accessibility_caption"]
                            if(img_description.find(":") !=-1):
                                img_description=img_description[img_description.index(":")+1:]
                            else:
                                img_description="Null"
                        else:
                            img_description="Video"
                        posts_data.append({'Tag_Id': tag_id,'Tag_Name': tag_name,'Extraction_Date':currentDT,'Posts_W_Hashtag':number_of_post_with_hashtag,'Owner_Id': owner_id,'Shortcode': shortcode,'Timestamp': timestamp,'Likes': likes,'Hashtags': hashtags,'Mentions': mentions,'Com\'s' : number_coms ,'Text': text,'Img_Description' : img_description,'Url_Img' : url_img})
                        r.status_code=404
                        posts_retrieved+=1
                        if (posts_retrieved%10==0):
                            print(posts_retrieved," Posts retrieved")
                        time_limit=time.time()
            pages_retrieved+=1
            page_time_end=time.time()
            #print("Page number :",pages_retrieved," retrieved - ","Time :",page_time_end-page_time_start)
        else :
            failed_requests+=1
                
    #print("Requests successful : ",successful_requests)
    #print("Requests failed : ",failed_requests)
    print("Kicked requests : ",kicked_requests)
    
    with open("#"+outfile_name+'_posts_data.json', 'w') as outfile:
        json.dump(posts_data, outfile) # save to json
    print("#"+outfile_name+'_posts_data.json has been created')
 
    #post = posts_data[0].keys()
   # with open("#"+outfile_name+'_posts_data.csv', 'w') as outfile:
    #    dict_writer = csv.DictWriter(outfile, post)
    #    dict_writer.writeheader()
    #    dict_writer.writerows(posts_data)


    print("Process finished")

# We can show here the html code of an user's page just to show breifly what it looks like

In [0]:
url = "https://www.instagram.com/univdauphine/"

r = requests.get(url,timeout = 5)
print(r.text)

<!DOCTYPE html>
<html lang="en" class="no-js not-logged-in client-root">
    <head>
        <meta charset="utf-8">
        <meta http-equiv="X-UA-Compatible" content="IE=edge">

        <title>
Université Paris Dauphine-PSL (@univdauphine) • Instagram photos and videos
</title>

        
        <meta name="robots" content="noimageindex, noarchive">
        <meta name="apple-mobile-web-app-status-bar-style" content="default">
        <meta name="mobile-web-app-capable" content="yes">
        <meta name="theme-color" content="#ffffff">
        <meta id="viewport" name="viewport" content="width=device-width, initial-scale=1, minimum-scale=1, maximum-scale=1, viewport-fit=cover">
        
        
    <link rel="manifest" href="/data/manifest.json">

        <link rel="preload" href="/static/bundles/metro/ConsumerUICommons.css/8cf3deeebba8.css" as="style" type="text/css" crossorigin="anonymous" />
<link rel="preload" href="/static/bundles/metro/ConsumerAsyncCommons.css/92d523fcf698.css" a

# Here is the function that collect datas for posts of a certain user (see report for more information)


In [0]:
def Posts_Data_From_User(user,nb_posts_max,nb_likes_min,outfile_name):
    end_cursor = '' # empty for the 1st page
    print("Start retrieving posts data from User: ",user,)
    successful_requests=0
    failed_requests=0
    kicked_requests=0
    kicked_limit=10
    posts_data=[]
    currentDT = datetime.datetime.now()
    currentDT_month=str(currentDT.month)
    currentDT_day=str(currentDT.day)
    pages_retrieved = 0
    if (len(currentDT_month)==1):
        currentDT_month="0"+currentDT_month
    if (len(currentDT_day)==1):
        currentDT_day="0"+currentDT_day
    currentDT=currentDT_day+currentDT_month+str(currentDT.year)
    url = "https://www.instagram.com/{0}/?__a=1".format(user)
    kicked_times=0
    while True and kicked_times<kicked_limit:
        try:
            r = requests.get(url,timeout = 5)
        except Exception as e:
            time.sleep(1)
            kicked_requests+=1
            kicked_times+=1
            continue
        break
    if ( r.status_code == 200):
        raw_data = r.text
        data = json.loads(raw_data.strip(),strict = False)
        #print(data)
        user_dic=data['graphql']['user']
        edge_owner_to_timeline_media=user_dic['edge_owner_to_timeline_media']
        end_cursor = edge_owner_to_timeline_media ['page_info']['end_cursor'] # value for the next page
        edges = edge_owner_to_timeline_media['edges'] # list with posts
        retrieved_post_number=0
        page_time_start=time.time()
        user_name=user_dic['username']
        user_id=user_dic['id']
        followers=user_dic['edge_followed_by']['count']
        number_of_posts=edge_owner_to_timeline_media['count']
        is_verified=user_dic['is_verified']
        for node in edges:
            if(len(posts_data)<nb_posts_max):
                #print("Retrieving post number",retrieved_post_number, "from page ",i)
                post=node['node']
                likes=post['edge_liked_by']['count']
                if(likes>= nb_likes_min):
                    owner_id = user_id
                    timestamp = post['taken_at_timestamp']
                    shortcode=post['shortcode']
                    try :
                        number_coms=post['edge_media_to_comment']['count'] 
                    except :
                        number_coms=post['edge_media_parent_to_comment']['count']

                    coms=[]

                    url_img=post['display_url'] 
                    edges_media=post['edge_media_to_caption']['edges']
                    if (len(edges_media)>0):
                        text=edges_media[0]['node']['text'] 
                    else :
                        text=""

                    hashtags=Hashtags_From_Text(text)
                    mentions=Mentions_From_Text(text)
                    if( post["is_video"]==False):
                        img_description=post["accessibility_caption"]
                        print(img_description)
                        if(img_description != None):
                            img_description=img_description
                        else:
                            img_description="Null"
                    else:
                        img_description="video"
                    posts_data.append({'User_Id': user_id,'User_Name': user_name,'Is_Verified' : is_verified,'Followers' : followers,'Extraction_Date':currentDT,'Nb_Of_Posts':number_of_posts,'Owner_Id': owner_id,'Shortcode': shortcode,'Timestamp': timestamp,'Likes': likes,'Hashtags': hashtags,'Mentions': mentions,'Com\'s' : number_coms ,'Text': text,'Img_Description' : img_description,'Url_Img' : url_img})
                    time.sleep(5) # insurence to not reach a time limit
                    r.status_code=404
                    retrieved_post_number+=1
        url ="https://www.instagram.com/graphql/query/?query_hash=f2405b236d85e8296cf30347c9f08c2a&variables=%7B%22id%22%3A%22{0}%22%2C%22first%22%3A{1}%2C%22after%22%3A%22QVFDQnFGLTNFeXRpaVdUbU4zam1IQzczS1pZUDJhbTk5dDFBMl9TdGJYUDB4Y2VSUlBlMTFFS0xlbWlnUHlLRjdSeXRsRXp5bWJQdEJWQnVrZUc1eGRvUw%3D%3D%22%7D".format(user_id,number_of_posts)    
        kicked_times=0
    while True and kicked_times<kicked_limit:
        try:
            r = requests.get(url,timeout = 5)
        except Exception as e:
            time.sleep(1)
            kicked_requests+=1
            kicked_times+=1
            continue
        break
    if ( r.status_code == 200):
        raw_data = r.text
        datas = json.loads(raw_data.strip(),strict = False)
        edges = datas['data']['user']['edge_owner_to_timeline_media']['edges']
        for node in edges:
            if(len(posts_data)<nb_posts_max):
                #print("Retrieving post number",retrieved_post_number, "from page ",i)
                post=node['node']
                likes=post['edge_media_preview_like']['count'] 
                if(likes>= nb_likes_min):
                    owner_id = user_id
                    timestamp = post['taken_at_timestamp']
                    shortcode=post['shortcode']
                    try :
                        number_coms=post['edge_media_to_comment']['count'] 
                    except :
                        number_coms=post['edge_media_parent_to_comment']['count']

                    coms=[]

                    url_img=post['display_url'] 
                    edges_media=post['edge_media_to_caption']['edges']
                    if (len(edges_media)>0):
                        text=edges_media[0]['node']['text'] 
                    else :
                        text=""

                    hashtags=Hashtags_From_Text(text)
                    mentions=Mentions_From_Text(text)
                    posts_data.append({'User_Id': user_id,'User_Name': user_name,'Is_Verified' : is_verified,'Followers' : followers,'Extraction_Date':currentDT,'Nb_Of_Posts':number_of_posts,'Owner_Id': owner_id,'Shortcode': shortcode,'Timestamp': timestamp,'Likes': likes,'Hashtags': hashtags,'Mentions': mentions,'Com\'s' : number_coms ,'Text': text,'Url_Img' : url_img})
                    r.status_code=404
                    retrieved_post_number+=1
    else :
        failed_requests+=1

    print("Kicked requests : ",kicked_requests)
    post = posts_data[0].keys()

    #with open(outfile_name+'_posts_data.csv', 'w') as outfile:
    #    dict_writer = csv.DictWriter(outfile, post)
     #   dict_writer.writeheader()
     #   dict_writer.writerows(posts_data)

    with open(outfile_name+'_posts_data.json', 'w') as outfile:
        json.dump(posts_data, outfile) # save to json
    print(outfile_name+'_posts_data.json has been created')
    df=pd.read_json(outfile_name+"_posts_data.json")
    df.to_csv(outfile_name+'_data_results.csv')
    print(outfile_name+'_data_results.csv has been created')
    print("Process finished")
    
#END CURSOR A REGLER
#POSTS A PLUSIEURS IMAGES

# These are 2 functions for text processing 

In [0]:
def Hashtags_From_Text(txt):
    hashtags=[]
    index=0
    count=0
    while(txt.find("#",index,len(txt))!=-1):
        start_index = txt.index("#",index,len(txt))+1
        if ( start_index<len(txt)):
            char = txt[start_index]
            char_index=start_index
            hashtag=""
            while(char_index<len(txt) and txt[char_index] not in [" ",".","#","\\"]):
                hashtag+=txt[char_index]
                char_index+=1
            hashtags.append(hashtag)
            index=char_index
        else :
            start_index+=1
            index=start_index
    return hashtags
            
def Mentions_From_Text(txt):
    hashtags=[]
    index=0
    count=0
    while(txt.find("@",index,len(txt))!=-1):
        start_index = txt.index("@",index,len(txt))+1
        if ( start_index<len(txt)):
            char = txt[start_index]
            char_index=start_index
            hashtag=""
            while(char_index<len(txt) and txt[char_index] not in [" ",".","#","\\","@"]):
                hashtag+=txt[char_index]
                char_index+=1
            hashtags.append(hashtag)
            index=char_index
        else :
            start_index+=1
            index=char_index
    return hashtags
        

# Let's try it on a Hashtag : \#datawrangling

In [0]:
tag="datawrangling"
outfile_name = tag
number_of_posts=10
minimum_likes_on_posts = 10
request_time_limit_between_2_posts = (minimum_likes_on_posts+1)*1

t1=time.time()
Posts_Data_From_Tag(tag,number_of_posts,outfile_name,minimum_likes_on_posts,request_time_limit_between_2_posts)
t2=time.time()
print(t2-t1,' seconds')

df = pd.read_json("#"+outfile_name+'_posts_data.json')
df

Start retrieving posts data from hashtag:  datawrangling
Number of posts requested:  10
10  Posts retrieved
Kicked requests :  0
#datawrangling_posts_data.json has been created
Process finished
1.5664196014404297  seconds


,Tag_Id,Tag_Name,Extraction_Date,Posts_W_Hashtag,Owner_Id,Shortcode,Timestamp,Likes,Hashtags,Mentions,Com's,Text,Img_Description,Url_Img
0,17843831179043460,datawrangling,21032020,862,10602884403,B9zqLYug8bL,2020-03-16 20:11:07,39,"[datatechcon\nNeed, dashboard, kpi, smallbusin...",[datatechcon],0,Customers are the foundation of any business’ ...,Video,https://scontent-atl3-1.cdninstagram.com/v/t51...
1,17843831179043460,datawrangling,21032020,862,9299626406,B9yFEDcAKx5,2020-03-16 05:18:48,27,"[businessintelligence\n, automateddatawranglin...",[],4,Why wouldn’t you prefer the speed and data pro...,Video,https://scontent-atl3-1.cdninstagram.com/v/t51...
2,17843831179043460,datawrangling,21032020,862,13935806959,B9o5bH1gczi,2020-03-12 15:42:41,12,"[datawrangling, businessintelligence, inglês, ...",[],0,#datawrangling #businessintelligence #inglês #...,"1 person, text",https://scontent-atl3-1.cdninstagram.com/v/t51...
3,17843831179043460,datawrangling,21032020,862,9299626406,B9nlb2lAtH_,2020-03-12 03:31:24,14,[],[],1,Reduce human resource costs by taking complete...,Video,https://scontent-atl3-1.cdninstagram.com/v/t51...
4,17843831179043460,datawrangling,21032020,862,12763554440,B9gIW5oAt9N,2020-03-09 06:00:01,35,"[ml, machinelearningalgorithms, machinelearnin...","[data, data, data]",1,A very subtle and sharp way to understand the ...,possible text that says 'Statistics: Given th...,https://scontent-atl3-1.cdninstagram.com/v/t51...
5,17843831179043460,datawrangling,21032020,862,12763554440,B9eJomnAI0P,2020-03-08 11:32:41,32,[],"[data, data, data]",1,Some very cool and handy R packages which help...,Null,https://scontent-atl3-1.cdninstagram.com/v/t51...
6,17843831179043460,datawrangling,21032020,862,186032245,B9ZVgKvBMRe,2020-03-06 14:40:12,49,"[CineColombiano, detrasdecamaras, datawrangling]","[ficcifestival,]",0,Después de un tiempo dando vueltas por el mund...,"1 person, standing and text",https://scontent-atl3-1.cdninstagram.com/v/t51...
7,17843831179043460,datawrangling,21032020,862,9299626406,B9TD-UGgv50,2020-03-04 04:11:35,10,[],[],1,Change reality into a more desirable state wit...,"1 person, beard and outdoor, possible text th...",https://scontent-atl3-1.cdninstagram.com/v/t51...
8,17843831179043460,datawrangling,21032020,862,18075761281,B9RXXdHHKji,2020-03-03 12:22:33,10,"[data, datascience, datasavy, datascientist, d...",[],0,Data visualization tools... #data#datascience#...,Null,https://scontent-atl3-1.cdninstagram.com/v/t51...
9,17843831179043460,datawrangling,21032020,862,27474528070,B9HfkqQH0s-,2020-02-28 16:21:51,15,"[fridayfeeling, fridaytreats, ditpic, davincir...",[],1,Cheeky Friday beer to round off the weeks work...,Null,https://scontent-atl3-1.cdninstagram.com/v/t51...


# Now we try on an user : @univdauphine

In [0]:
user="univdauphine"
outfile_name = user
number_of_posts=20
nb_likes_min = 10

t1=time.time()
Posts_Data_From_User(user,number_of_posts,nb_likes_min,outfile_name)
t2=time.time()
print(t2-t1,' seconds')

df = pd.read_csv(outfile_name+'_data_results.csv')
df

Start retrieving posts data from User:  univdauphine
Photo by Université Paris Dauphine-PSL on March 20, 2020. Image may contain: sky, grass, outdoor and text
Photo by Université Paris Dauphine-PSL in Université Paris Dauphine - PSL. Image may contain: sky, cloud and outdoor
None
Photo by Université Paris Dauphine-PSL on November 19, 2019. Image may contain: one or more people
Photo by Université Paris Dauphine-PSL in Université Paris Dauphine - PSL. Image may contain: one or more people, people standing, tree and outdoor
Photo by Université Paris Dauphine-PSL in Université Paris Dauphine - PSL. Image may contain: one or more people, crowd and indoor
Photo by Université Paris Dauphine-PSL on September 27, 2019. Image may contain: one or more people and indoor
Photo by Université Paris Dauphine-PSL on September 13, 2019. Image may contain: one or more people, crowd and outdoor
Photo by Université Paris Dauphine-PSL in Université Paris Dauphine - PSL. Image may contain: outdoor
Photo by 

,Unnamed: 0,User_Id,User_Name,Is_Verified,Followers,Extraction_Date,Nb_Of_Posts,Owner_Id,Shortcode,Timestamp,Likes,Hashtags,Mentions,Com's,Text,Img_Description,Url_Img
0,0,257662767,univdauphine,False,6868,21032020,283,257662767,B99yqg6oJxi,2020-03-20 18:27:40,58,['Covid19]'],[],0,[#Covid19] Dauphine est fermée jusqu’à nouvel ...,"é Paris Dauphine-PSL on March 20, 2020. Image ...",https://scontent-atl3-1.cdninstagram.com/v/t51...
1,1,257662767,univdauphine,False,6868,21032020,283,257662767,B7s9C8soicn,2020-01-24 12:28:31,436,"['JPO\n\nVenez', 'Repost', 'assoetudiante', 'v...","['qutbusiness', 'univdauphine', 'itsabrina']",8,Orientation 📣 Dauphine organise sa Journée des...,é Paris Dauphine-PSL in Université Paris Dauph...,https://scontent-atl3-1.cdninstagram.com/v/t51...
2,2,257662767,univdauphine,False,6868,21032020,283,257662767,B7YHb_zHKZG,2020-01-16 10:19:19,423,[],"['univdauphine', 'figaroetudiant']",10,🏆 Vous avez classé @univdauphine en 2e place d...,video,https://scontent-atl3-1.cdninstagram.com/v/t51...
3,3,257662767,univdauphine,False,6868,21032020,283,257662767,B7EPCLhCon_,2020-01-08 16:56:50,450,['parcoursup'],[],34,"Nouveauté 2020 📢 Dès le 22 janvier, postulez e...",Null,https://scontent-atl3-1.cdninstagram.com/v/t51...
4,4,257662767,univdauphine,False,6868,21032020,283,257662767,B5DM9JxoSb9,2019-11-19 14:16:38,255,"['SEEPH2019]', 'SemaineduHandicap', 'DauphineD...",[],1,[#SEEPH2019] Handi'cap sur Dauphine ! L'univer...,"é Paris Dauphine-PSL on November 19, 2019. Ima...",https://scontent-atl3-1.cdninstagram.com/v/t51...
5,5,257662767,univdauphine,False,6868,21032020,283,257662767,B3_s2JCAwhi,2019-10-24 09:06:57,225,"['DauphineDurable', 'ODD']",[],11,"Un projet, une idée ou une initiative concerna...",é Paris Dauphine-PSL in Université Paris Dauph...,https://scontent-atl3-1.cdninstagram.com/v/t51...
6,6,257662767,univdauphine,False,6868,21032020,283,257662767,B3PeofXohs0,2019-10-05 15:39:12,716,[],[],4,Félicitations aux près de 1400 diplômées et di...,é Paris Dauphine-PSL in Université Paris Dauph...,https://scontent-atl3-1.cdninstagram.com/v/t51...
7,7,257662767,univdauphine,False,6868,21032020,283,257662767,B269Z0ECofG,2019-09-27 16:24:02,445,"['HousingByDauphine,']",['dauphine-housing'],5,"Hier soir, Isabelle Huault, Présidente de Daup...","é Paris Dauphine-PSL on September 27, 2019. Im...",https://scontent-atl3-1.cdninstagram.com/v/t51...
8,8,257662767,univdauphine,False,6868,21032020,283,257662767,B2WpRlPCevD,2019-09-13 13:55:29,606,"['RentréeDauphine', 'LSO', 'assoetudiante', 'v...","['univdauphine', 'psl_univ']",9,#RentréeDauphine des L1 #LSO \nL’occasion de d...,"é Paris Dauphine-PSL on September 13, 2019. Im...",https://scontent-atl3-1.cdninstagram.com/v/t51...
9,9,257662767,univdauphine,False,6868,21032020,283,257662767,B0YVFkhiESV,2019-07-26 12:34:46,303,"['summer', 'paris', 'dauphine', 'psl', 'univer...","['univdauphine', 'psl_univ']",1,[Fermeture estivale] Le campus parisien de @un...,é Paris Dauphine-PSL in Université Paris Dauph...,https://scontent-atl3-1.cdninstagram.com/v/t51...
